## Black Box Approach for all 10 genres classification

We aim at training a Neural Network to distinguish between all the different genres.
In this model we try and augment the dataset

In [1]:
#importing google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#setting the working directory
%cd /gdrive/MyDrive/polimi/NAML/NAML_proj/

/gdrive/MyDrive/polimi/NAML/NAML_proj


In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import random
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import librosa
import os

tfk = tf.keras
tfkl = tf.keras.layers
seed = 42

The dataset is composed of Mel-Spectrograms of each audio sample, labeled with respect to each genre

In [4]:
dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
        'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
n_genres = 10

for genre, genre_number in genres.items():
    for filename in os.listdir(f'dataset_old/genres/{genre}'):
        songname = f'dataset_old/genres/{genre}/{filename}'
        for i in range(3):
          y, sr = librosa.load(songname, mono=True, duration=9.9, offset = i*9.9)
          ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512)
          ps = librosa.power_to_db(ps**2)
          dataset.append( (ps, genre_number) )
    print(str(genre+' done'))

blues done
classical done
country done
disco done
hiphop done
jazz done
metal done
pop done
reggae done
rock done


We split the dataset according to the following composition:


*   70% training set
*   20% validation set
*   10% test set

Maintaining equal proportions amongst classes



In [51]:
order = np.arange(start = 0, stop = 100, step = 1)

np.random.seed(seed)

training = []
validation = []
test = []

for i in range(n_genres):
  shuffle = np.random.permutation(order)
  for k in range(70):
    for kk in range(3):
      training.append(dataset[i*100 + shuffle[k]*3 + kk])
  for l in range(20):
    for ll in range(3):
      validation.append(dataset[i*100 + shuffle[l+70]*3 + ll])
  for m in range(10):
    for mm in range(3):
      test.append(dataset[i*100 + shuffle[m+90]*3 + mm])

In [52]:
X_train, Y_train = zip(*training)
X_valid, Y_valid = zip(*validation)
X_test, Y_test = zip(*test)

X_train = np.array([x.reshape( (128, 853, 1) ) for x in X_train])
X_valid = np.array([x.reshape( (128, 853, 1) ) for x in X_valid])
X_test = np.array([x.reshape( (128, 853, 1) ) for x in X_test])


Y_train = np.array(tfk.utils.to_categorical(Y_train, n_genres))
Y_valid = np.array(tfk.utils.to_categorical(Y_valid, n_genres))
Y_test = np.array(tfk.utils.to_categorical(Y_test, n_genres))

The proposed model is composed as a stack of convolutional layer, followed by a Global Average Pooling layer leading to a fully connected section.

Optimization is performed as a Batch version of Adam optimizer algorithm

In [56]:
def build_model(input_shape, n_units):
# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        name = 'conv_1',
        filters=4,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv1_2 = tfkl.Conv2D(
        name = 'conv_1_2',
        filters=8,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        name = 'pool_1',
        pool_size = (2, 2)
    )(conv1_2)

    conv2 = tfkl.Conv2D(
        name = 'conv_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv2_2 = tfkl.Conv2D(
        name = 'conv_2_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv2)
    pool2 = tfkl.MaxPooling2D(
        name = 'pool_2',
        pool_size = (2, 2)
    )(conv2_2)

    conv3 = tfkl.Conv2D(
        name = 'conv_3',
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        name = 'pool_3',
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        name = 'conv_4',
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        name = 'pool_4',
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        name = 'conv_5',
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        name = 'pool_5',
        pool_size = (2, 2)
    )(conv5)

    conv6 = tfkl.Conv2D(
        name = 'conv_6',
        filters=286,
        kernel_size=(1, 1),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    global_average = tfkl.GlobalAveragePooling2D(name = 'GAP')(conv6)
    global_average = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_GAP')(global_average)
    
    classifier_layer = tfkl.Dense(units=64, name='Classifier', activation='relu')(global_average)
    
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier')(classifier_layer)
    classifier_layer_2 = tfkl.Dense(units=32, name='Classifier_2', activation='relu')(classifier_layer)
    classifier_layer_2 = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier_2')(classifier_layer_2)
    output_layer = tfkl.Dense(units=n_units, activation='softmax', name='Output')(classifier_layer_2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model

    return model

In [59]:
input_shape = (128, 853, 1)

model = build_model(input_shape, n_genres)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 128, 853, 1)]     0         
                                                                 
 conv_1 (Conv2D)             (None, 128, 853, 4)       104       
                                                                 
 conv_1_2 (Conv2D)           (None, 128, 853, 8)       296       
                                                                 
 pool_1 (MaxPooling2D)       (None, 64, 426, 8)        0         
                                                                 
 conv_2 (Conv2D)             (None, 64, 426, 16)       1168      
                                                                 
 conv_2_2 (Conv2D)           (None, 64, 426, 16)       2320      
                                                                 
 pool_2 (MaxPooling2D)       (None, 32, 213, 16)       0     

In [60]:
metrics_accuracy = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []

n_trials = 10
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
adaptive_LR = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)

for i in range(n_trials):
  model = build_model(input_shape, n_genres)
  standard_history = model.fit(
    x = X_train,
    y = Y_train,
    epochs = 500,
    batch_size = 100,
    validation_data= (X_valid, Y_valid),
    callbacks = [early_stopping, adaptive_LR]
    )
  predictions = model.predict(X_test)
  metrics_accuracy.append(accuracy_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1)))
  metrics_precision.append(precision_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_recall.append(recall_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_f1.append(f1_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  model.save(str('/gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_'+str(i)))

Epoch 1/500
21/21 [==============================] - 8s 337ms/step - loss: 1.9960 - accuracy: 0.2529 - val_loss: 1.6388 - val_accuracy: 0.3017 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 7s 314ms/step - loss: 1.5718 - accuracy: 0.3605 - val_loss: 1.1237 - val_accuracy: 0.5850 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 7s 320ms/step - loss: 1.3049 - accuracy: 0.4776 - val_loss: 1.0169 - val_accuracy: 0.5967 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 7s 318ms/step - loss: 1.1510 - accuracy: 0.5286 - val_loss: 0.9158 - val_accuracy: 0.6200 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 7s 323ms/step - loss: 1.0137 - accuracy: 0.5790 - val_loss: 0.7495 - val_accuracy: 0.7633 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 7s 322ms/step - loss: 0.9181 - accuracy: 0.6362 - val_loss: 0.7042 - val_accuracy: 0.7367 - lr: 0.0010
Epoch 7/500
21/21 [==============================] - 7s 319ms/st

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_0/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_0/assets


Epoch 1/500
21/21 [==============================] - 8s 342ms/step - loss: 2.4743 - accuracy: 0.2133 - val_loss: 1.6658 - val_accuracy: 0.2883 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 7s 320ms/step - loss: 1.7352 - accuracy: 0.3195 - val_loss: 1.3820 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 7s 317ms/step - loss: 1.4744 - accuracy: 0.4333 - val_loss: 1.1337 - val_accuracy: 0.5367 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 7s 321ms/step - loss: 1.2732 - accuracy: 0.4900 - val_loss: 1.0579 - val_accuracy: 0.5433 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 7s 321ms/step - loss: 1.1548 - accuracy: 0.5214 - val_loss: 0.9453 - val_accuracy: 0.5483 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 7s 322ms/step - loss: 1.0837 - accuracy: 0.5505 - val_loss: 0.8777 - val_accuracy: 0.5433 - lr: 0.0010
Epoch 7/500
21/21 [==============================] - 7s 317ms/st

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_1/assets


...

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_8/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_8/assets


Epoch 1/500
21/21 [==============================] - 8s 332ms/step - loss: 2.0463 - accuracy: 0.2476 - val_loss: 1.6213 - val_accuracy: 0.3017 - lr: 0.0010
Epoch 2/500
21/21 [==============================] - 7s 320ms/step - loss: 1.7122 - accuracy: 0.2824 - val_loss: 1.4727 - val_accuracy: 0.3183 - lr: 0.0010
Epoch 3/500
21/21 [==============================] - 7s 322ms/step - loss: 1.5230 - accuracy: 0.3752 - val_loss: 1.1980 - val_accuracy: 0.5283 - lr: 0.0010
Epoch 4/500
21/21 [==============================] - 7s 320ms/step - loss: 1.2822 - accuracy: 0.4710 - val_loss: 1.0189 - val_accuracy: 0.5367 - lr: 0.0010
Epoch 5/500
21/21 [==============================] - 7s 316ms/step - loss: 1.1370 - accuracy: 0.5119 - val_loss: 0.9084 - val_accuracy: 0.5383 - lr: 0.0010
Epoch 6/500
21/21 [==============================] - 7s 321ms/step - loss: 1.0715 - accuracy: 0.5262 - val_loss: 0.9076 - val_accuracy: 0.5517 - lr: 0.0010
Epoch 7/500
21/21 [==============================] - 7s 317ms/st

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_9/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/augmented_0_9/assets


In [61]:
metrics_accuracy = np.array(metrics_accuracy)
metrics_precision = np.array(metrics_precision)
metrics_recall = np.array(metrics_recall)
metrics_f1 = np.array(metrics_f1)

print(str('mean accuracy = ' + str(metrics_accuracy.mean() )))
print(str('mean precision = ' + str(metrics_precision.mean() )))
print(str('mean recall = ' + str(metrics_recall.mean() )))
print(str('mean f1 = ' + str(metrics_f1.mean() )))

print('***___***')

print(str('accuracy standard deviation = ' + str(metrics_accuracy.std() )))
print(str('precision standard deviation = ' + str(metrics_precision.std() )))
print(str('recall standard deviation = ' + str(metrics_recall.std() )))
print(str('f1 standard deviation = ' + str(metrics_f1.std() )))

print('***___***')

print(str('accuracy range = ' + str(metrics_accuracy.max() - metrics_accuracy.min() )))
print(str('precision range = ' + str(metrics_precision.max() - metrics_precision.min() )))
print(str('recall range = ' + str(metrics_recall.max() - metrics_recall.min() )))
print(str('f1 range = ' + str(metrics_f1.max() - metrics_f1.min() )))

print('***___***')

print(str('min accuracy = ' + str(metrics_accuracy.min() )))
print(str('min precision = ' + str(metrics_precision.min() )))
print(str('min recall = ' + str(metrics_recall.min() )))
print(str('min f1 = ' + str(metrics_f1.min() )))



mean accuracy = 0.9823333333333334
mean precision = 0.9825343356525365
mean recall = 0.9807945041816011
mean f1 = 0.9814570733892328
***___***
accuracy standard deviation = 0.013169830843425614
precision standard deviation = 0.013875828423890185
recall standard deviation = 0.012280660460940554
f1 standard deviation = 0.01305181968758055
***___***
accuracy range = 0.050000000000000044
precision range = 0.05132805221463932
recall range = 0.04735023041474662
f1 range = 0.04943363493460673
***___***
min accuracy = 0.9466666666666667
min precision = 0.9441264932399062
min recall = 0.9486815156169994
min f1 = 0.9462727870837419


In [62]:
print(metrics_accuracy)
print(metrics_precision)
print(metrics_recall)
print(metrics_f1)

[0.99       0.99       0.98666667 0.98       0.99666667 0.98
 0.94666667 0.98666667 0.97666667 0.99      ]
[0.99001976 0.99187705 0.9893617  0.98031377 0.99545455 0.98039196
 0.94412649 0.98736019 0.97775936 0.98867854]
[0.98809524 0.98937532 0.98412698 0.97738095 0.99603175 0.97671958
 0.94868152 0.98412698 0.97597286 0.98743386]
[0.98886161 0.99053232 0.9863685  0.97832075 0.99570642 0.97837636
 0.94627279 0.98542555 0.97667297 0.98803347]
